In [1]:
import os
import requests
from bs4 import BeautifulSoup
from PIL import Image
from urllib.parse import urljoin
from io import BytesIO


def save_google_images(search_terms, num_images, save_folder):
    # Create the save folder if it doesn't exist
    os.makedirs(save_folder, exist_ok=True)

    for search_term in search_terms:
        # Create a folder for the current search term
        search_folder = os.path.join(save_folder, search_term)
        os.makedirs(search_folder, exist_ok=True)

        # Prepare the search URL
        search_url = f"https://www.google.com/search?q={search_term}&tbm=isch"

        # Send a GET request to the search URL
        response = requests.get(search_url)
        response.raise_for_status()

        # Parse the response content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all image elements on the page
        image_elements = soup.find_all('img')

        # Iterate over the image elements and save the thumbnails
        count = 0
        for img in image_elements:
            # Get the thumbnail URL
            thumbnail_url = img['src']
            if thumbnail_url.startswith('/'):
                # Handle relative URLs by joining with the base URL
                thumbnail_url = urljoin(search_url, thumbnail_url)

            # Send a GET request to the thumbnail URL
            thumbnail_response = requests.get(thumbnail_url)
            thumbnail_response.raise_for_status()

            # Open the thumbnail image with PIL
            image = Image.open(BytesIO(thumbnail_response.content))

            # Convert the image to RGB mode
            image = image.convert("RGB")

            # Resize the image to the desired size
            image = image.resize((150, 150))

            # Save the resized image in the search term folder
            filename = f"{search_folder}/{search_term}_{count}.jpg"
            image.save(filename)

            count += 1
            if count == num_images:
                break

        print(f"{count} thumbnail images saved for '{search_term}' in {search_folder}")


# Example usage
search_terms = ["underbench oven", "kitchen exhaust", "kitchen island", "double sink"]
num_images = 20
save_folder = "thumbnails"

save_google_images(search_terms, num_images, save_folder)



20 thumbnail images saved for 'underbench oven' in thumbnails\underbench oven
20 thumbnail images saved for 'kitchen exhaust' in thumbnails\kitchen exhaust
20 thumbnail images saved for 'kitchen island' in thumbnails\kitchen island
20 thumbnail images saved for 'double sink' in thumbnails\double sink
